# Airbnb Listings ETL
This notebook performs an **Extract, Transform, Load (ETL)** process on Airbnb listings data.

## 1. Import Libraries
We begin by importing necessary Python libraries for data manipulation and analysis.

In [179]:
import re
import pandas as pd
import numpy as np

## 2. Load the Data
We read the Airbnb listings dataset from a CSV file. The separator used in the file is `;`, which needs to be specified.

In [180]:
air_bnb_data = pd.read_csv(r"C:\Users\isabe\PycharmProjects\GSB 520 ETL\Data\air-bnb-listings.csv",
                           sep = ";")

## 3. Preview the Data
We display the first few rows to understand the dataset structure.

In [181]:
air_bnb_data.head()

,Room ID,Name,Host ID,Neighbourhood,Room type,Room Price,Minimum nights,Number of reviews,Date last review,Number of reviews per month,Rooms rent by the host,Availibility,Updated Date,City,Country,Coordinates,Location
0,9153958,.2 La Casa sui Tetti dell'Oltrarno,47655975,Centro Storico,Entire home/apt,50,1,363,2020-02-17,6.69,2,0,2020-06-19,Florence,Italy,"43.76678908298443, 11.24589003597409","Italy, Florence, Centro Storico"
1,9191572,Firenze Studio Porta Romana,47816872,Centro Storico,Entire home/apt,70,2,46,2018-03-15,0.83,1,364,2020-06-19,Florence,Italy,"43.76544742217545, 11.24254982481698","Italy, Florence, Centro Storico"
2,9306481,ELEGANT FLAT FLORENCE CITY CENTER,42112573,Centro Storico,Entire home/apt,120,2,132,2020-01-29,2.42,1,270,2020-06-19,Florence,Italy,"43.779967029995476, 11.261314413194068","Italy, Florence, Centro Storico"
3,9524225,Luxury Modern Studio - Florence,11065786,Campo di Marte,Entire home/apt,75,3,56,2019-12-30,1.01,5,364,2020-06-19,Florence,Italy,"43.76815640280208, 11.275230547227133","Italy, Florence, Campo di Marte"
4,9602552,Florence is a dream,956926,Campo di Marte,Entire home/apt,50,2,24,2018-10-27,0.44,1,365,2020-06-19,Florence,Italy,"43.78809311664643, 11.273347846180846","Italy, Florence, Campo di Marte"


## 4. Analyze Country Distribution
We count the number of listings per country.

In [182]:
air_bnb_data["Country"].value_counts()

Country
United states     240662
Italy             179607
Spain             109219
United kingdom    103964
France             90621
Australia          89580
China              88885
Greece             57646
Canada             56659
Ireland            35996
Germany            35985
Portugal           35965
Brazil             35731
Denmark            28523
Argentina          24134
South africa       24062
Turkey             23728
Mexico             21824
Netherlands        19450
Chile              15970
Japan              14715
Austria            12974
Belgium            12808
Czech republic     12565
Norway              8830
Taiwan              8103
Switzerland         7894
Sweden              7635
Singapore           7323
Belize              2960
Name: count, dtype: int64

## 5. Filter down to U.S. Data
We filter the listings down to those located in the United States. 

In [183]:
us_airbnb = air_bnb_data[air_bnb_data["Country"] == "United states"]

## 6. Process Location Data
We extract latitude and longitude from the 'Coordinates' column and convert them to numerical values.

In [184]:
# Splitting the 'Coordinates' column into 'Latitude' and 'Longitude'
us_airbnb[['Latitude', 'Longitude']] = us_airbnb['Coordinates'].str.split(', ', expand=True)

# Convert Latitude and Longitude to float for numerical operations
us_airbnb.loc[:, 'Latitude'] = us_airbnb['Latitude'].astype(float)
us_airbnb.loc[:, 'Longitude'] = us_airbnb['Longitude'].astype(float)

# Dropping the original 'Coordinates' column
us_airbnb.drop(columns=['Coordinates'], inplace=True)

C:\Users\isabe\AppData\Local\Temp\ipykernel_24900\1967335914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_airbnb[['Latitude', 'Longitude']] = us_airbnb['Coordinates'].str.split(', ', expand=True)
C:\Users\isabe\AppData\Local\Temp\ipykernel_24900\1967335914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_airbnb[['Latitude', 'Longitude']] = us_airbnb['Coordinates'].str.split(', ', expand=True)
C:\Users\isabe\AppData\Local\Temp\ipykernel_24900\1967335914.py:9: SettingWithCopyWarning: 
A value 

## 7. Standardize City Names
We clean and standardize city names by replacing hyphens with spaces, capitalizing abbreviations, and removing unnecessary state abbreviations.

In [187]:
# Clean the city names: replace hyphens with spaces and convert to title case
us_airbnb['City'] = us_airbnb['City'].str.replace("-", " ").str.title()

# Define a mapping for abbreviations that should be fully capitalized.
abbrev_map = {
    'Dc': 'DC',
    'Nv': 'NV',
    'Msa': 'MSA',
    'Or': 'OR'
}

def capitalize_abbrev(city):
    # Split the city name into words
    words = city.split()
    # Replace any word found in our abbreviation map with its uppercase version
    words = [abbrev_map.get(word, word) for word in words]
    # Rejoin the words into a string
    return " ".join(words)

# Apply the function to the cleaned city names
us_airbnb['City'] = us_airbnb['City'].apply(capitalize_abbrev)

# Adjust specific city names (e.g., "New York City" to "New York")
def adjust_city_name(city):
    if city == "New York City":
        return "New York"
    return city

us_airbnb['City'] = us_airbnb['City'].apply(adjust_city_name)

def drop_state(city):
    """
    Remove a trailing state abbreviation (a space followed by 2+ uppercase letters)
    from the city name unless the city is "Washington DC".
    """
    # If the city is "Washington DC", keep it as-is (case-insensitive comparison)
    if city.strip().lower() == "washington dc":
        return "Washington, D.C."
    # Otherwise, remove trailing state abbreviation
    return re.sub(r'\s+[A-Z]{2,}$', '', city)

# Apply the function to the 'City' column and store the result in a new column
us_airbnb["City"] = us_airbnb["City"].apply(drop_state)

C:\Users\isabe\AppData\Local\Temp\ipykernel_24900\3224347339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_airbnb['City'] = us_airbnb['City'].str.replace("-", " ").str.title()
C:\Users\isabe\AppData\Local\Temp\ipykernel_24900\3224347339.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_airbnb['City'] = us_airbnb['City'].apply(capitalize_abbrev)
C:\Users\isabe\AppData\Local\Temp\ipykernel_24900\3224347339.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

## 8. Check for Missing Values
We check for any missing values in the dataset.

In [189]:
us_airbnb.isna().sum().sort_values(ascending=False)

Number of reviews per month    49937
Date last review               49937
Name                              32
Room ID                            0
Host ID                            0
Neighbourhood                      0
Room Price                         0
Room type                          0
Number of reviews                  0
Minimum nights                     0
Rooms rent by the host             0
Availibility                       0
Updated Date                       0
City                               0
Country                            0
Location                           0
Latitude                           0
Longitude                          0
dtype: int64

## 9. Handle Missing Data
We fill or adjust missing values for specific columns.

In [190]:
# Ensure we're working on a copy to avoid warnings
final_airbnb_df = us_airbnb.copy()

# Fill missing values for 'Number of reviews per month'
final_airbnb_df['Number of reviews per month'] = final_airbnb_df['Number of reviews per month'].fillna(0)

# Convert 'Date last review' to datetime and handle NaN values
final_airbnb_df['Date last review'] = pd.to_datetime(final_airbnb_df['Date last review'])
final_airbnb_df['Date last review'] = final_airbnb_df['Date last review'].fillna(pd.NaT)

# Fill missing 'Name' values with a placeholder
final_airbnb_df['Name'] = final_airbnb_df['Name'].fillna("Unnamed Listing")

## 10. Save the Cleaned Dataset
Finally, we save the cleaned dataset for further analysis.

In [191]:
# Save Cleaned Dataset
final_airbnb_df.to_csv(
    r"C:\Users\isabe\PycharmProjects\GSB 520 ETL\Clean Data\cleaned_airbnb_data.csv", index=False)
